# Implement variable elimination in the following model

The Student Network

![alt text](sb.png)

In [2]:
# Define the product of two tables:

In [3]:
dificulty = {0: 0.6, 1: 0.4}
intelligence = {0: 0.7, 1: 0.3}
sat = {0: {0: 0.95, 1:0.2}, 1: {0: 0.05, 1: 0.8}}
grade = {1: {(0, 0): 0.3, (0, 1): 0.05, (1, 0): 0.9, (1,1):0.5}, 2: {(0, 0): 0.4, (0, 1): 0.25, (1, 0): 0.08, (1,1):0.3}, 3: {(0, 0): 0.3, (0, 1): 0.7, (1, 0): 0.02, (1,1):0.2}}

In [6]:
#def multiply(A, idx_commA, B, idx_commB):
#    # (i, d, g, l)
#    for t in A:
#        n_i = A[t]*B[]

In [7]:
grade = {(0,0,1):0.3, (0,0,2):0.4, (0,0,3):0.3, 
        (0,1,1):0.05, (0,1,2):0.25, (0,1,3):0.7,
        (1,0,1): 0.9, (1,0,2):0.08, (1,0,3):0.02,
        (1,1,1):0.5, (1,1,2):0.3, (1,1,3):0.2}

In [8]:
intelligence = {(1, 0): 0.1, (1, 1):0.9, (2,0):0.4, (2,1):0.6,
                (3,0):0.99, (3,1):0.01}

In [9]:
intelligence

{(1, 0): 0.1,
 (1, 1): 0.9,
 (2, 0): 0.4,
 (2, 1): 0.6,
 (3, 0): 0.99,
 (3, 1): 0.01}

In [10]:
grade

{(0, 0, 1): 0.3,
 (0, 0, 2): 0.4,
 (0, 0, 3): 0.3,
 (0, 1, 1): 0.05,
 (0, 1, 2): 0.25,
 (0, 1, 3): 0.7,
 (1, 0, 1): 0.9,
 (1, 0, 2): 0.08,
 (1, 0, 3): 0.02,
 (1, 1, 1): 0.5,
 (1, 1, 2): 0.3,
 (1, 1, 3): 0.2}

In [276]:
import numpy as np

In [279]:
diffic = np.array([0.6, 0.4])
intell = np.array([0.7, 0.3])
sat = np.array([[0.95, 0.05],[0.2, 0.8]])
grade = np.array([[0.3, 0.4, 0.3],[0.05, 0.25, 0.7],[0.9, 0.08, 0.02], [0.5, 0.3, 0.2]])
letter = np.array([[0.1, 0.9],[0.4, 0.6], [0.99, 0.01]])

In [282]:
letter[0]

array([0.1, 0.9])

In [265]:
dificulty = {0: 0.6, 1: 0.4}
intelligence = {0: 0.7, 1: 0.3}
sat = {0: {0: 0.95, 1:0.2}, 1: {0: 0.05, 1: 0.8}}
grade = {1: {0}}

In [274]:
tableSAT = {0:{}, 1:{}}
for fromTab in intelligence:
    for toTab in sat:
        tableSAT[toTab][fromTab] = intelligence[fromTab]*sat[toTab][fromTab]

In [275]:
tableSAT

{0: {0: 0.6649999999999999, 1: 0.06}, 1: {0: 0.034999999999999996, 1: 0.24}}

In [ ]:
tableG = {0:{}, 1:{}}
for fromTab in intelligence:
    for toTab in sat:
        tableG[toTab][fromTab] = intelligence[fromTab]*sat[toTab][fromTab]

In [1]:
myd = {}
myd[0,1] = "hola"

In [2]:
myd[0,1]

'hola'

In [12]:
class Node:
    def __init__(self, states, patterns):
        self.states = states
        self.patterns = patterns
        self.table = {}
        self.currentState = 0
    def pushInTable(self, data, indexes):
        if len(self.patterns) > 0:
            n_inputs = len(self.patterns)
            if n_inputs == 1:
                self.table[indexes[0]] = data
            elif n_inputs == 2:
                #print(indexes)
                self.table[(indexes[0],indexes[1])] = data
        else:
            self.table = data

In [209]:
class Graph:
    def __init__(self, nodes = {}):
        self.nodes = nodes
        self.evidences = {}
        self.querys = set()
        self.hidden = set()
        self.conjunt = set()
    def addNode(self, name, states, patterns = []):
        self.nodes[name] = Node(states, patterns)
        self.conjunt.add(name)
        #self.nodes.append(Node(states, patterns))
    def addData(self, name, data, ind = []):
        self.nodes[name].pushInTable(data, indexes = ind)
    
    def setEvidence(self, name, state_idx):
        self.evidences[name] = state_idx
    def setQueries(self, name):
        self.querys.add(name)
    
    def setHidden(self):
        if self.querys.isdisjoint(set(self.evidences)):
            self.hidden = self.conjunt - set(self.evidences) - self.querys
        else:
            print("Error, Evidence set & Query Set are not disjoint, please Check!\n")
        
    def StractStates(self, name, querystate):
        states = []
        if name in self.evidences:
            states.append(self.evidences[name])
        elif name in self.hidden:
            states = self.nodes[name].states
        elif name in self.querys:
            states.append(querystate)
        
        return states
    def Multiplication(self, father, child):
        import copy
        tab = copy.deepcopy(child.table)
        if len(child.patterns) == 1:
            for idx in father.states:
                for i in range(len(child.table[idx])):
                    tab[idx][i] = child.table[idx][i]*father.table[idx]
        
        elif len(child.patterns) == 2:
            posIdx = 0
            # Get in what position is the current parent
            for ii in range(len(child.patterns)):
                if child.patterns[ii] == father:
                    posIdx = ii
            ## ****
            if posIdx == 0:
                for sf1 in father.states:
                    for sf2 in child.patterns[1].states:
                        for i in child.states:
                            tab[(sf1, sf2)][i] = father.table[sf1]*child.table[(sf1, sf2)][i]
            else:
                for sf1 in child.patterns[0].states:
                    for sf2 in father.states:
                        for i in child.states:
                            tab[(sf1, sf2)][i] = father.table[sf2]*child.table[(sf1, sf2)][i]
        
        return tab
    
    def transformToTable(self, table):
        tab = {}
        for t in table:
            for s in range(len(table[t])):
                L = list(t)
                L.insert(0, s)
                tab[tuple(L)] = table[t][s]
        return tab
    
    def Marginalize(self, table, index):
        tab = {}
        for t in table:
            tn = list(t)
            del(tn[index])
            if len(tn) == 1:
                tn = tn[0]
            else:
                tn = tuple(tn)
            try: 
                tab[tn] = tab[tn] + table[t]
            except:
                tab[tn] = table[t]
        
        return tab                
                

    def RunVariableElimination(self, stateQuery = 0):
        ## For the moment it formula just will work with 1 node as Query.
        # Hidden are factors:
        

        
    


### Create a Graphical Model

In [210]:
graph = Graph()

In [211]:
# Define the node Difficult
graph.addNode('Difficult', states = [0, 1])

In [212]:
graph.addData('Difficult', data = [0.6, 0.4])

In [213]:
# Define the node Intelligence
graph.addNode('Intelligence', states = [0, 1])

In [214]:
graph.addData('Intelligence', data = [0.7, 0.3])

In [215]:
# Define the node SAT
graph.addNode('SAT', states = [0, 1], patterns=[graph.nodes['Intelligence']])

In [216]:
graph.addData('SAT', ind=[0], data=[0.95, 0.05])
graph.addData('SAT', ind=[1], data=[0.2, 0.8])

In [217]:
# Define the node Grades:
graph.addNode('Grade', states = [0, 1, 2], patterns=[graph.nodes['Difficult'], graph.nodes['Intelligence']])

In [218]:
graph.addData('Grade', data = [0.3, 0.4, 0.3],ind = [0, 0])
graph.addData('Grade', data = [0.05, 0.25, 0.7],ind = [0, 1])
graph.addData('Grade', data = [0.9, 0.08, 0.02],ind = [1, 0])
graph.addData('Grade', data = [0.5, 0.3, 0.2],ind = [1, 1])

In [219]:
graph.nodes['Grade'].table

{(0, 0): [0.3, 0.4, 0.3],
 (0, 1): [0.05, 0.25, 0.7],
 (1, 0): [0.9, 0.08, 0.02],
 (1, 1): [0.5, 0.3, 0.2]}

In [220]:
graph.nodes['SAT'].table

{0: [0.95, 0.05], 1: [0.2, 0.8]}

In [221]:
# Define the node Letter
graph.addNode('Letter', states = [0, 1], patterns = [graph.nodes['Grade']])

In [222]:
graph.addData('Letter', data = [0.1, 0.9], ind = [0])
graph.addData('Letter', data = [0.4, 0.6], ind = [1])
graph.addData('Letter', data = [0.99, 0.01], ind = [2])

In [223]:
graph.nodes['Letter'].table

{0: [0.1, 0.9], 1: [0.4, 0.6], 2: [0.99, 0.01]}

In [224]:
graph.nodes['Grade'].table

{(0, 0): [0.3, 0.4, 0.3],
 (0, 1): [0.05, 0.25, 0.7],
 (1, 0): [0.9, 0.08, 0.02],
 (1, 1): [0.5, 0.3, 0.2]}

In [225]:
graph.Multiplication(graph.nodes['Intelligence'], graph.nodes['SAT'])

{0: [0.6649999999999999, 0.034999999999999996], 1: [0.06, 0.24]}

In [226]:
tt = graph.Multiplication(graph.nodes['Intelligence'], graph.nodes['Grade'])

In [227]:
tt

{(0, 0): [0.21, 0.27999999999999997, 0.21],
 (0, 1): [0.015, 0.075, 0.21],
 (1, 0): [0.63, 0.055999999999999994, 0.013999999999999999],
 (1, 1): [0.15, 0.09, 0.06]}

In [228]:
graph.transformToTable(tt)

{(0, 0, 0): 0.21,
 (0, 0, 1): 0.015,
 (0, 1, 0): 0.63,
 (0, 1, 1): 0.15,
 (1, 0, 0): 0.27999999999999997,
 (1, 0, 1): 0.075,
 (1, 1, 0): 0.055999999999999994,
 (1, 1, 1): 0.09,
 (2, 0, 0): 0.21,
 (2, 0, 1): 0.21,
 (2, 1, 0): 0.013999999999999999,
 (2, 1, 1): 0.06}

In [229]:
graph.nodes['Letter'].table

{0: [0.1, 0.9], 1: [0.4, 0.6], 2: [0.99, 0.01]}

In [230]:
graph.setEvidence('SAT', 1)

In [231]:
graph.setQueries('Intelligence')

In [232]:
graph.setHidden()

In [259]:
graph.evidences={'Letter':1, 'SAT':1}

In [260]:
graph.querys={'Intelligence'}

In [261]:
graph.setHidden()

In [262]:
graph.hidden

{'Difficult', 'Grade'}

In [263]:
graph.RunVariableElimination(0)

[0, 1]


0.00029400000000000004

# Example

Get the $P(SAT | Intelligence = 1)$

In [243]:
graph.Multiplication(graph.nodes['Intelligence'], graph.nodes['SAT'])

{0: [0.6649999999999999, 0.034999999999999996], 1: [0.06, 0.24]}

In [244]:
# the probability that Get the sat given the intelligence

In [245]:
0.24/(0.06 + 0.24)

0.8